# Ling 380 - Week 13

# Topic modelling

Topic modelling is a very useful technique to get information from a large dataset. It is a type of *unsupervised machine learning*. Recall that *supervised machine learning* involved learning patterns from data, given a dataset and labels (e.g., movie reviews and stars). In unsupervised machine learning, we find patterns, but we do not have labels associated with the data. The task is to learn to classify or cluster the data by exploiting the patterns or similarities in the documents.

The basic idea behind topic modelling is that you can identify topics or themes in a collection of documents using words that co-occur. One of the implementations of this idea is LDA (Latent Dirichlet Allocation), which assumes a distribution that we try to find using words in the text. The figure below is from an easy to follow paper outlining LDA. 

<figure>
    <img src="./img/Blei_topic_modelling.png" style="width:450px;"
         alt="supervised classification">
</figure>

Diagram from: Blei, D. M. (2012). [Probabilistic topic models](https://www.cs.columbia.edu/~blei/papers/Blei2012.pdf). Communications of the ACM, 55(4), 77-84. 

The data for topic modelling needs to be normalized following the usual steps we have done so far: tokenization, lemmatization, and stopword removal. Then, we use a topic modelling module to find the topics in the data. 

The implementation here is based on a project on extracting topics from news articles, [TACT](https://github.com/sfu-discourse-lab/TACT/tree/master). 

If you want to see topic models in action, go to the [research site](https://gendergaptracker.research.sfu.ca/apps/topicmodel) for the Gender Gap Tracker and check topics for any month in the last 8 years. Those are the main topics covered in Canadian news. 

## Data

We will work with a collection of news articles, a part of the [SFU Opinion and Comments Corpus (SOCC)](https://github.com/sfu-discourse-lab/SOCC). The corpus was collected in our lab, the Discourse Processing Lab, for a project on evaluative language in online news comments. It consists of: opinion articles, comments, and annotated comments from the Canadian newspaper _The Globe and Mail_. We'll work with the articles, which should be in the data directory. If not, you can always download the corpus directly from the page above or from its [Kaggle page](https://www.kaggle.com/datasets/mtaboada/sfu-opinion-and-comments-corpus-socc) and save the `gnm_articles.csv` file to your data directory.

## Install gensim and import statements

In [1]:
# run only once
!pip install gensim --user

In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import gensim
from gensim import corpora
from gensim.models import LdaModel

## Import and examine the data

In [3]:
df = pd.read_csv('./data/gnm_articles.csv', encoding = 'utf-8')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10339 entries, 0 to 10338
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   article_id           10339 non-null  int64  
 1   title                10339 non-null  object 
 2   article_url          10339 non-null  object 
 3   author               10339 non-null  object 
 4   published_date       10339 non-null  object 
 5   ncomments            10339 non-null  float64
 6   ntop_level_comments  10339 non-null  float64
 7   article_text         10339 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 646.3+ KB


In [5]:
df.head()

,article_id,title,article_url,author,published_date,ncomments,ntop_level_comments,article_text
0,26842506,The Tories deserve another mandate - Stephen H...,http://www.theglobeandmail.com/opinion/editori...,GLOBE EDITORIAL,2015-10-16 EDT,2187.0,1378.0,<p>All elections are choices among imperfect a...
1,26055892,Harper hysteria a sign of closed liberal minds,http://www.theglobeandmail.com/opinion/harper-...,Konrad Yakabuski,2015-08-24 EDT,1103.0,455.0,<p>If even a fraction of the darkness that his...
2,6929035,Too many first nations people live in a dream ...,http://www.theglobeandmail.com/opinion/too-man...,Jeffrey Simpson,2013-01-05 EST,1164.0,433.0,<p>Large elements of aboriginal Canada live in...
3,19047636,The Globe's editorial board endorses Tim Hudak...,http://www.theglobeandmail.com/opinion/editori...,GLOBE EDITORIAL,2014-06-06 EDT,905.0,432.0,"<p>Over four days, The Globe editorial board l..."
4,11672346,Disgruntled Arab states look to strip Canada o...,http://www.theglobeandmail.com/news/world/disg...,Campbell Clark,2013-05-02 EDT,1129.0,411.0,<p>Growing discontent among Arab nations over ...


## Define a function to process the article_text column

The only information we are interested in is in the `article_text` column, which contains the body of the articles. We will reuse and modify the function from last week to clean up that data. We will:

* Remove the html tokens (the text has `<p>` and `</p>` to mark paragraphs)
* Tokenize and lowercase
* Remove stopwords
* Lemmatize -- this one is added from last week; we use the WordNet lemmatizer

We apply this function to the original df, to create a new column, `article_processed`. Then, we create a dictionary from that blob of text. You can also first extract the `article_text` to a string and process it, then create the dictionary (like we did in Week 12). 

In [6]:
# define a function that: 1. tokenizes, 2. lowercases and
# 3. lemmatizes

# create the lemmatizer first
lemmatizer = WordNetLemmatizer()

# define the function
def process_text(text):
    clean_text = re.sub(r'<.*?>', '', text)
    words = word_tokenize(clean_text.lower())
    cleaned_words = [word for word in words if word.isalpha() and word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in cleaned_words]
    return lemmatized_words

In [7]:
df['article_processed'] = df['article_text'].apply(process_text)

In [8]:
df.head()

,article_id,title,article_url,author,published_date,ncomments,ntop_level_comments,article_text,article_processed
0,26842506,The Tories deserve another mandate - Stephen H...,http://www.theglobeandmail.com/opinion/editori...,GLOBE EDITORIAL,2015-10-16 EDT,2187.0,1378.0,<p>All elections are choices among imperfect a...,"[election, choice, among, imperfect, alternati..."
1,26055892,Harper hysteria a sign of closed liberal minds,http://www.theglobeandmail.com/opinion/harper-...,Konrad Yakabuski,2015-08-24 EDT,1103.0,455.0,<p>If even a fraction of the darkness that his...,"[even, fraction, darkness, hater, say, descend..."
2,6929035,Too many first nations people live in a dream ...,http://www.theglobeandmail.com/opinion/too-man...,Jeffrey Simpson,2013-01-05 EST,1164.0,433.0,<p>Large elements of aboriginal Canada live in...,"[large, element, aboriginal, canada, live, int..."
3,19047636,The Globe's editorial board endorses Tim Hudak...,http://www.theglobeandmail.com/opinion/editori...,GLOBE EDITORIAL,2014-06-06 EDT,905.0,432.0,"<p>Over four days, The Globe editorial board l...","[four, day, globe, editorial, board, looked, o..."
4,11672346,Disgruntled Arab states look to strip Canada o...,http://www.theglobeandmail.com/news/world/disg...,Campbell Clark,2013-05-02 EDT,1129.0,411.0,<p>Growing discontent among Arab nations over ...,"[growing, discontent, among, arab, nation, har..."


## Parameters for gensim

Now that we have clean text, we will process it with gensim functions. The first one creates a dictionary of the words in the text. The second creates a corpus, a bag of words with the frequency of all the words. Finally, the LdaModel actually creates the topic model.

There are several parameters you can set with LDA models. For instance, you may want to filter out words that have very few instances, because they are only representative of one or two articles. In this case, what we are setting as parameters are the number of topics and the number of iterations. We train the model with 10 topics, that is, we assume that there are 10 topics across the entire dataset. And we do 15 passes. 

Once we have extracted the topics, we can inspect the most representative X words in each topic. In this case, I set it to 20. One thing you could do is to try and label the topics. So, for instance, these are the two top topics when I run the model:

* Topic 0: party, liberal, conservative, election, government, harper, minister, political, would, leader, ndp, campaign, prime, new, one, vote, voter, trudeau, could, time

* Topic 1: per, cent, government, year, tax, canada, would, health, care, money, system, budget, cost, canadian, one, rate, spending, education, province, country

I could rename them to: 

* Topic 0 = Canadian politics
* Topic 1 = Budget and government spending

The labels in the [Gender Gap Tracker](https://gendergaptracker.research.sfu.ca/apps/topicmodel) site are done manually every month. 

In [9]:
# use the gensim function to create a dictionary of the words in the text
dictionary = corpora.Dictionary(df['article_processed'])

In [10]:
# you can inspect the contents of that dictionary
for token, token_id in list(dictionary.token2id.items())[:10]:
    print('{} => {}'.format(token, token_id))

acceptance => 0
accomplishment => 1
achieves => 2
action => 3
almost => 4
also => 5
alternative => 6
among => 7
anyone => 8
anything => 9


In [11]:
# use the doc2bow function to create a corpus, a bag of words (bow) of the text and the word counts
corpus = [dictionary.doc2bow(text) for text in df['article_processed']]

In [12]:
# create the LDA model
lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

In [ ]:
# print the top 20 words for each topic
topics = lda_model.print_topics(num_words=20)
for topic_num, topic in topics:
    print(f"Topic {topic_num}: ", end="")
    words = topic.split(' + ')
    word_list = [word.split('*')[1].strip('\"') for word in words] 
    print(", ".join(word_list))
    print()

Topic 0: city, vancouver, woman, one, year, game, team, street, new, sport, hockey, would, two, home, building, toronto, place, say, player, montreal

Topic 1: party, liberal, conservative, election, quebec, leader, political, ndp, campaign, would, harper, voter, vote, new, minister, one, candidate, trudeau, politics, could

Topic 2: child, woman, health, people, canada, school, canadian, per, student, cent, university, care, need, many, system, education, family, one, year, work

Topic 3: canada, canadian, oil, china, country, government, world, would, trade, new, energy, company, economic, global, year, market, pipeline, industry, foreign, development

Topic 4: city, mayor, ford, toronto, first, transit, aboriginal, nation, council, people, said, police, would, subway, councillor, chief, say, community, could, one

Topic 5: one, people, like, time, day, say, know, get, thing, even, said, year, think, u, would, way, could, good, book, make

Topic 6: government, province, tax, year, pe

In [14]:
# you can also print the weight of each word in the topic
for topic_num, topic in topics:
    print("Topic {}:".format(topic_num), end=" ")
    words = topic.split(' + ')
    word_weight_list = ["{} ({:.4f})".format(term.strip('\"'), float(weight)) for weight, term in (word.split('*') for word in words)]
    print(", ".join(word_weight_list))
    print()

Topic 0: city (0.0120), vancouver (0.0110), woman (0.0100), one (0.0060), year (0.0060), game (0.0060), team (0.0050), street (0.0050), new (0.0050), sport (0.0050), hockey (0.0050), would (0.0040), two (0.0040), home (0.0040), building (0.0030), toronto (0.0030), place (0.0030), say (0.0030), player (0.0030), montreal (0.0030)

Topic 1: party (0.0250), liberal (0.0150), conservative (0.0140), election (0.0140), quebec (0.0100), leader (0.0100), political (0.0090), ndp (0.0090), campaign (0.0080), would (0.0080), harper (0.0080), voter (0.0070), vote (0.0070), new (0.0060), minister (0.0050), one (0.0050), candidate (0.0050), trudeau (0.0050), politics (0.0040), could (0.0040)

Topic 2: child (0.0090), woman (0.0090), health (0.0090), people (0.0080), canada (0.0070), school (0.0070), canadian (0.0070), per (0.0070), student (0.0060), cent (0.0060), university (0.0060), care (0.0060), need (0.0060), many (0.0050), system (0.0050), education (0.0050), family (0.0050), one (0.0050), year

# Summary

We have learned about topic modelling and how to extract topics from data using the LDA model in the gensim library. 